<a href="https://colab.research.google.com/github/martinpius/GANS/blob/main/DCGAN_Implementantion_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch
  print(f">>>> You are on CoLaB with torch version {torch.__version__}")
except Exception as e:
  print(f">>>> {type(e)}: {e}\n>>>> please correct {type(e)} and reload")
  COLAB = False
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
def time_fmt(t: float = 239.198)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"{h} hrs: {m:>02} min: {s:>05.2f} sec"
print(f">>>> time formating\tplease wait.....\n>>>> time elapsed\t{time_fmt()}")

Mounted at /content/drive
>>>> You are on CoLaB with torch version 1.8.1+cu101
>>>> time formating	please wait.....
>>>> time elapsed	0 hrs: 03 min: 59.00 sec


In [2]:
#In this notbook we are going to train a DCGAN network from scratch. Demonstration will be on mnist-dataset

In [17]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import numpy as np
import math, time, sys, os, random
import PIL, copy
from torch.utils.tensorboard import SummaryWriter
from albumentations.torch import ToTensor
from tqdm import tqdm


In [18]:
#Setup the default seeds for reproducability:
seed = 1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [19]:
#DCGAN is mainly builts of CNN architecture with both Convolutions 
#(for the discriminator) and de-convolution layers for the generator

In [20]:
#The discriminator: This class will compose a discriminator network

In [21]:
class Discriminator(nn.Module):
  def __init__(self, img_channels, d_features):
    super(Discriminator, self).__init__()
    self.discriminator = nn.Sequential(
        nn.Conv2d(img_channels, d_features, kernel_size = 4, stride = 2, padding = 1),
        nn.LeakyReLU(0.2),
        self.__dblock__(d_features, 2*d_features, 4, 2, 1),
        self.__dblock__(2*d_features, 4*d_features, 4, 2, 1),
        self.__dblock__(4*d_features, 8*d_features, 4, 2, 1),
        nn.Conv2d(8*d_features, 1, kernel_size = 4, stride = 2, padding = 0),
        nn.Sigmoid())
  
  def __dblock__(self, in_features, out_features, kernel_size, stride, padding):
    return nn.Sequential(
        nn.Conv2d(in_features,
                  out_features,
                  kernel_size,
                  stride,
                  padding,
                  bias = False),
    nn.BatchNorm2d(out_features),
    nn.LeakyReLU(0.2))
  
  def forward(self, input_tensor):
    return self.discriminator(input_tensor)


In [22]:
#The Generator class(This is the network which use de-convolution layers to reconstruct fake image from the random noise)

In [23]:
class Generator(nn.Module):
  def __init__(self, img_channels, z_dim, g_features):
    super(Generator, self).__init__()
    self.generator = nn.Sequential(
        self.__gblock__(z_dim, g_features *16, 4, 1, 0),
        self.__gblock__(16*g_features, 8*g_features, 4, 2, 1),
        self.__gblock__(8*g_features, 4*g_features, 4, 2, 1),
        self.__gblock__(4*g_features, 2*g_features, 4, 2, 1),
        nn.ConvTranspose2d(2*g_features, img_channels, kernel_size = 4, stride = 2, padding =1),
        nn.Tanh())
  def __gblock__(self, in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.ConvTranspose2d(
            in_channels, out_channels, kernel_size, stride, padding, bias = False
        ),
        nn.BatchNorm2d(out_channels),
        nn.ReLU())
  
  def forward(self, input_tensor):
    return self.generator(input_tensor)

In [24]:
#Initialize the weights of the network to random normal distribution with mean 0 and std 0.02(like in the paper)

In [25]:
def __initializer__(model):
  for m in model.modules():
    if isinstance(m,(nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
      nn.init.normal_(m.weight.data, mean = 0.0, std = 0.02)

In [26]:
#Testing the netwok if its gives the desired output:
def __test__():
  z_dim = 100
  img_channels = 3
  W = 64
  H = 64
  batch_size = 64
  disc = Discriminator(img_channels, 8)
  gen = Generator(img_channels, z_dim, 8)
  noise = torch.randn(batch_size, z_dim,1,1)
  rand_img = torch.randn(batch_size, img_channels, H, W)
  gen_out = gen(noise)
  disc_out = disc(rand_img)
  return f"gen_out_shape: {gen_out.shape}, disc_out_shape: {disc_out.shape}"


In [27]:
__test__()

'gen_out_shape: torch.Size([64, 3, 64, 64]), disc_out_shape: torch.Size([64, 1, 1, 1])'

In [28]:
#Model Hyper-parameters:
EPOCHS = 5
d_features = 64
g_features = 64
batch_size = 128
z_dim = 100
img_channels = 1
img_size = 64
learning_rate = 2e-4
generator = Generator(img_channels,z_dim, g_features).to(device)
discriminator = Discriminator(img_channels, d_features).to(device)
__initializer__(generator)
__initializer__(discriminator)
print(generator)
print("\n\n")
print(discriminator)

Generator(
  (generator): Sequential(
    (0): Sequential(
      (0): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(1, 1), bias=False)
      (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (2): Sequential(
      (0): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (3): Sequential(
      (0): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (4): ConvTranspose2d(12

In [29]:
#Downloading and preprocess the data from torchvision:
transforms = transforms.Compose([
                                 transforms.Resize(img_size),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.5 for _ in range(img_channels)],
                                                      [0.5 for _ in range(img_channels)])
])

dfm = datasets.MNIST(root = "mnist_dcgan/", transform = transforms, download = True)
loader = DataLoader(dataset = dfm, batch_size = batch_size, shuffle = True)
x_loader, y_loader = next(iter(loader))
print(f"x_loader_shape: {x_loader.shape}\ty_loader_shape: {y_loader.shape}")


Extracting mnist_dcgan/MNIST/raw/train-images-idx3-ubyte.gz to mnist_dcgan/MNIST/raw




Extracting mnist_dcgan/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_dcgan/MNIST/raw




Extracting mnist_dcgan/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_dcgan/MNIST/raw




Extracting mnist_dcgan/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_dcgan/MNIST/raw

Processing...
Done!
x_loader_shape: torch.Size([128, 1, 64, 64])	y_loader_shape: torch.Size([128])


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [33]:
#We can now build our training loop as follows:
step = 0
fake_writer = SummaryWriter(f"runs/fake_images")
real_writer = SummaryWriter(f"runs/real_images")
fixed_noise = torch.randn(32, z_dim, 1,1).to(device)
disc_opt = optim.Adam(params = discriminator.parameters(), lr = learning_rate, betas = (0.5, 0.999))
gen_opt = optim.Adam(params = generator.parameters(), lr = learning_rate, betas = (0.5, 0.999))
loss = nn.BCELoss()


In [34]:
global_tic = time.time()
for epoch in range(EPOCHS):
  tic = time.time()
  print(f">>>> The begining of epoch {epoch + 1}\n>>>> please wait while the model is training........")
  for idx, (real, _) in enumerate(tqdm(loader)):
    real = real.to(device)
    fake = torch.randn(batch_size, z_dim, 1, 1).to(device)
    noise = generator(fake)
    #training the discriminator
    real_out = discriminator(real).reshape(-1)
    fake_out = discriminator(noise).reshape(-1)
    real_loss = loss(real_out, torch.ones_like(real_out))
    fake_loss = loss(fake_out, torch.zeros_like(fake_out))
    disc_loss = (real_loss + fake_loss) / 2
    discriminator.zero_grad()
    disc_loss.backward(retain_graph = True)
    disc_opt.step()

    #training the generator (maximizing (log(D(G(z)))))
    gen_out = discriminator(noise).reshape(-1)#forward pass
    gen_loss = loss(gen_out, torch.ones_like(gen_out)) #fooling the discriminator
    generator.zero_grad()#initialize the gradients to zeros
    gen_loss.backward()#backward pass
    gen_opt.step() #gradient descent using Adam optimizer with 0.5 momentum
    toc = time.time()
    if idx % 200 == 0:
      print(f"\n>>>> time elapsed at the end of epoch {epoch + 1} for batch {idx} is {time_fmt(toc - tic)}")
      print(f">>>> generator loss: {gen_loss:.4f} | generator PPL: {math.exp(gen_loss):7.4f}")
      print(f">>>> discriminator loss: {disc_loss:.4f} | discriminator PPL: {math.exp(disc_loss):7.4f}")

      #recording real and fake images(generated with our DCGAN) to be vizualized in tensorboard
      with torch.no_grad():
        fake_images = generator(fixed_noise).to(device)
        img_grid_real = torchvision.utils.make_grid(real[:32], normalize = True)
        img_grid_fake = torchvision.utils.make_grid(fake_images[:32], normalize = True)
        real_writer.add_image("Real_imgs", img_grid_real, global_step = step)
        fake_writer.add_image("fake_imgs", img_grid_fake, global_step = step)
      step+=1 
global_toc = time.time()
print(f"\n>>>> time elapsed after 5 epochs training of this model is: {time_fmt(global_toc - global_tic)}")


  0%|          | 0/469 [00:00<?, ?it/s]

>>>> The begining of epoch 1
>>>> please wait while the model is training........

>>>> time elapsed at the end of epoch 1 for batch 0 is 0 hrs: 00 min: 00.00 sec


  0%|          | 1/469 [00:00<04:31,  1.72it/s]

>>>> generator loss: 0.9438 | generator PPL:  2.5697
>>>> discriminator loss: 0.5753 | discriminator PPL:  1.7776


 43%|████▎     | 200/469 [01:18<01:42,  2.62it/s]

...


>>>> time elapsed at the end of epoch 5 for batch 400 is 0 hrs: 02 min: 31.00 sec


 86%|████████▌ | 401/469 [02:31<00:34,  1.97it/s]

>>>> generator loss: 1.0271 | generator PPL:  2.7931
>>>> discriminator loss: 0.3760 | discriminator PPL:  1.4564


100%|██████████| 469/469 [02:56<00:00,  2.65it/s]


>>>> time elapsed after 5 epochs training of this model is: 0 hrs: 14 min: 54.00 sec


In [42]:
!tensorboard --logdir_spec runs --bind_all

2021-05-25 21:48:04.146167: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
TensorBoard 2.5.0 at http://2065c65a145f:6006/ (Press CTRL+C to quit)
